In [11]:
import geemap
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [31]:
feature = Map.draw_last_feature
ROI = feature.geometry()

ROI.getInfo()

{'geodesic': False,
 'type': 'Polygon',
 'coordinates': [[[-7.028509, 53.078172],
   [-7.028509, 53.123522],
   [-6.97082, 53.123522],
   [-6.97082, 53.078172],
   [-7.028509, 53.078172]]]}

In [32]:
import ee

def maskS2clouds(image):
    qa = image.select('QA60')
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
                qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask).divide(10000)

# Loading Sentinel-2 TOA reflectance data
imageCollection = ee.ImageCollection('COPERNICUS/S2') \
    .filterDate('2020-01-01', '2021-03-15') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5)) \
    .map(maskS2clouds) \
    .filterBounds(ROI) \
    .sort('CLOUDY_PIXEL_PERCENTAGE')

raw_image = ee.Image(imageCollection.toList(imageCollection.size()).get(0))

# Get the matrix for B2 band only
B2_image = geemap.ee_to_numpy(raw_image.select(['B2']), region=ROI)

In [35]:
import cv2

target_size = (50, 50)

B5_matrix = cv2.resize(B2_image, target_size, interpolation=cv2.INTER_NEAREST)
B5_matrix

array([[0.1398    , 0.14120001, 0.15279999, ..., 0.15260001, 0.1389    ,
        0.14480001],
       [0.1429    , 0.1473    , 0.1481    , ..., 0.15360001, 0.1534    ,
        0.1515    ],
       [0.14219999, 0.1523    , 0.1532    , ..., 0.1514    , 0.1407    ,
        0.1516    ],
       ...,
       [0.14560001, 0.1531    , 0.1508    , ..., 0.1428    , 0.13680001,
        0.1417    ],
       [0.14820001, 0.1496    , 0.1498    , ..., 0.14300001, 0.142     ,
        0.1411    ],
       [0.1521    , 0.14300001, 0.1402    , ..., 0.1408    , 0.1366    ,
        0.14229999]])